<a href="https://colab.research.google.com/github/YahyaEryani/quantum-model/blob/main/notebooks/04_MLP_Training_and_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Training, Tuning, and Evaluation
 In this notebook, we will train a multilayer perceptron (MLP) model on the Higgs boson dataset we have preprocessed in the `01_data_exploration` notebook. We will perform the model training and tuning process to obtain the best model with the highest accuracy possible.

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Loading Data

This code cell loads the training, validation, and test datasets that were saved in pickle format to the local directory.

In [8]:
# Mount Google Drive in Colab
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

# Load data from Google Drive
train_path = '/content/drive/MyDrive/Higgs_dataset/processed/training_data.pkl'
val_path   = '/content/drive/MyDrive/Higgs_dataset/processed/validation_data.pkl'
test_path  = '/content/drive/MyDrive/Higgs_dataset/processed/testing_data.pkl'

train_data = pd.read_pickle(train_path)
val_data = pd.read_pickle(val_path)
test_data = pd.read_pickle(test_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Prepare the data for training
This code separates the features and class labels from the train, validation, and test datasets.

In [9]:
# Separate features and labels
y_train = train_data['class_label']
X_train = train_data.drop('class_label', axis=1)
y_val = val_data['class_label']
X_val = val_data.drop('class_label', axis=1)
y_test = test_data['class_label']
X_test = test_data.drop('class_label', axis=1)

##Train the multilayer perceptron model
This code sets the hyperparameters for an MLP model, including the number of hidden layers, number of neurons per layer, activation functions, and dropout rates. It then trains the MLP model using the Adam optimizer with a specified learning rate, batch size, and number of epochs. Additionally, the model performance is monitored using the validation set during training.

In [10]:
# Define the MLP model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=256,
    validation_data=(X_val, y_val),
    verbose=1
)

Epoch 1/100
1407/1407 [==============================] - 8s 5ms/step - loss: 0.6413 - accuracy: 0.6257 - val_loss: 0.6062 - val_accuracy: 0.6781
Epoch 2/100
1407/1407 [==============================] - 7s 5ms/step - loss: 0.5961 - accuracy: 0.6809 - val_loss: 0.5720 - val_accuracy: 0.7041
Epoch 3/100
1407/1407 [==============================] - 7s 5ms/step - loss: 0.5749 - accuracy: 0.6978 - val_loss: 0.5546 - val_accuracy: 0.7122
Epoch 4/100
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5663 - accuracy: 0.7035 - val_loss: 0.5522 - val_accuracy: 0.7169
Epoch 5/100
1407/1407 [==============================] - 7s 5ms/step - loss: 0.5613 - accuracy: 0.7069 - val_loss: 0.5427 - val_accuracy: 0.7208
Epoch 6/100
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5584 - accuracy: 0.7096 - val_loss: 0.5438 - val_accuracy: 0.7221
Epoch 7/100
1407/1407 [==============================] - 7s 5ms/step - loss: 0.5559 - accuracy: 0.7112 - val_loss: 0.5413 - val_ac

## Make predictions on the test data and evaluate the model performance
This code uses the Histogram Gradient Boosting Classifier model that was previously trained to make predictions on the test data.

In [11]:
# Evaluate the model on the test set
y_test_pred = model.predict(X_test)
y_test_pred_classes = (y_test_pred > 0.5).astype(int)
test_accuracy = accuracy_score(y_test, y_test_pred_classes)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

3750/3750 [==============================] - 4s 1ms/step
Test Accuracy: 73.59%
